# North Carolina Scorecard

This notebook generates the county and statewide "future voter scorecards" for NY. It is generalized to be updated every month, with minimal changes.

Scorecard outputs (tables) are written back to BigQuery, where they are then read into Google Sheets for formatting

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq

## Inputs
Update the fields below each month

In [2]:
# Inputs
as_of_data_date = pd.Timestamp("2025-06-07")
as_of_election_date = None
acs_year = '2022' # 2022 for 2023 scorecards. ACS vintages trail by 2 years


## Outputs
Run the cells below, without edits each month

In [3]:
# Derived variables 

data_date_suffix = str(as_of_data_date.year) + str(as_of_data_date.month).rjust(2, "0") + str(as_of_data_date.day).rjust(2, "0")  # this is the rolling "as of" date, where we snap the line for 18yos

end_of_year_suffix = str(as_of_data_date.year) + str(12) # this is the "end-of-year" cohort date, where we alternatively snap the line for 18yos

# Define columns
    # ACS
EST_17_OR_18_OR_19_YO_THIS_YEAR = 'EST_17_OR_18_OR_19_YO_' + str(as_of_data_date.year) # col name for number of 17/18/19 yos in the "as of" year estimated from ACS
EST_45_PLUS_YO_THIS_YEAR = 'EST_45_PLUS_YO_' + str(as_of_data_date.year) # col name for number of 45+ yos in the "as of" year estimated from ACS


### County Scorecard

In [4]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
acs_S0101_table = "S0101_us_counties_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file by county, counting the number of registrants at a given age
* Then, left joins the county estimates for the total number of 17, 18, and 19yos from the ACS
    * The estimates for the total number of 17-19yo yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

* Ages are calculated as of the date of the voter file and as of the next general election 

In [5]:
str(as_of_data_date.day)

'7'

In [6]:
voter_file_table

'20250607_scorecard_nc'

In [7]:
sql = f"""
SELECT COUNT(*) 
FROM `tcc-research.nc_production.{voter_file_table}`
WHERE VOTER_STATUS IN ('A', 'I')
"""

In [8]:
# Define GCP project
project_id = "tcc-research"


# Define query, including variables and column names that adjust with time
as_of_date_str = as_of_data_date.strftime('%Y-%m-%d')

sql = f"""
WITH young_voters AS (
  SELECT  
    *,
    FLOOR(DATE_DIFF(
  DATE '2025-06-07', 
  DATE(CAST(YEAR_OF_BIRTH AS INT64), 1, 1), 
  DAY
) / 365.25) AS AGE_TODAY
  FROM `tcc-research.nc_production.{voter_file_table}`
  WHERE VOTER_STATUS IN ('ACTIVE', 'INACTIVE')
), young_voter_sum AS (
  SELECT
    COUNTY_FIPS,
    COUNTY_NAME,
    COUNTIF(AGE_TODAY = 18) AS N_VOTERS_18_AS_OF_{data_date_suffix},
    COUNTIF(AGE_TODAY = 19) AS N_VOTERS_19_AS_OF_{data_date_suffix},
    COUNTIF(AGE_TODAY >= 45) AS N_VOTERS_45_PLUS_AS_OF_{data_date_suffix}
  FROM young_voters
  GROUP BY COUNTY_FIPS, COUNTY_NAME
), acs_county AS (
  SELECT 
    LPAD(CAST(COUNTY_FIPS AS STRING), 5, '0') AS COUNTY_FIPS,
    EST_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS {EST_17_OR_18_OR_19_YO_THIS_YEAR},
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS {EST_45_PLUS_YO_THIS_YEAR}
  FROM `tcc-research.acs_sources.S0101_us_counties_acs5y_2022` 
  WHERE STATE_FIPS = '37'
)
SELECT
  young_voter_sum.*,
  acs_county.EST_15_TO_17_YO,
  acs_county.{EST_17_OR_18_OR_19_YO_THIS_YEAR},
  acs_county.{EST_45_PLUS_YO_THIS_YEAR}
FROM young_voter_sum
LEFT JOIN acs_county
  ON LPAD(CAST(young_voter_sum.COUNTY_FIPS AS STRING), 5, '0') = acs_county.COUNTY_FIPS
"""

# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [9]:
# Preview
df.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS_18_AS_OF_20250607,N_VOTERS_19_AS_OF_20250607,N_VOTERS_45_PLUS_AS_OF_20250607,EST_15_TO_17_YO,EST_17_OR_18_OR_19_YO_2025,EST_45_PLUS_YO_2025
0,059,DAVIE,132,461,21091,1693,564.333333,24638
1,197,YADKIN,81,347,15909,1415,471.666667,21399
2,193,WILKES,152,572,29017,2590,863.333333,38460
3,001,ALAMANCE,498,1966,67286,6617,2205.666667,85865
4,153,RICHMOND,181,472,17185,1882,627.333333,21534


In [10]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)


In [11]:
# Calculate estimated registration rate

N_VOTERS_18_AS_OF_DATA_DATE = 'N_VOTERS_18_AS_OF_'+ data_date_suffix # col name for estimated 18yo voters as of rolling date


EST_REG_RATE_18_YO_AS_OF_DATA_DATE = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

df_reg_est[EST_REG_RATE_18_YO_AS_OF_DATA_DATE] = df_reg_est[N_VOTERS_18_AS_OF_DATA_DATE] / df_reg_est[EST_17_OR_18_OR_19_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

df_reg_est.sort_values('EST_15_TO_17_YO', ascending=False)

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS_18_AS_OF_20250607,N_VOTERS_19_AS_OF_20250607,N_VOTERS_45_PLUS_AS_OF_20250607,EST_15_TO_17_YO,EST_17_OR_18_OR_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_RATE_18_YO_AS_OF_20250607
82,183,WAKE,4087,13339,431642,47558,15852.666667,499266,0.257812
14,119,MECKLENBURG,3431,11411,391201,42911,14303.666667,465789,0.239869
7,081,GUILFORD,1499,6330,210353,21093,7031.000000,253633,0.213199
81,067,FORSYTH,1129,4319,149611,15559,5186.333333,186737,0.217688
29,179,UNION,1157,3405,102626,12970,4323.333333,115787,0.267618
...,...,...,...,...,...,...,...,...,...
37,073,GATES,32,106,5096,349,116.333333,6345,0.275072
46,005,ALLEGHANY,33,76,5726,328,109.333333,6936,0.301829
71,075,GRAHAM,30,74,4031,297,99.000000,4616,0.30303
65,095,HYDE,6,33,2123,121,40.333333,2830,0.14876


#### Metric 3: Estimated registration rate of 45+ year olds as of a rolling date (i.e. latest month)


In [12]:
# Calculate estimated registration rate

N_VOTERS_45_PLUS_AS_OF_DATA_DATE = 'N_VOTERS_45_PLUS_AS_OF_'+ data_date_suffix # col name for estimated 18yo voters as of rolling date

EST_REG_RATE_45_PLUS_YO_AS_OF_DATA_DATE = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_DATA_DATE] = df_reg_est[N_VOTERS_45_PLUS_AS_OF_DATA_DATE] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

df_reg_est.sort_values(EST_45_PLUS_YO_THIS_YEAR, ascending=False)

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS_18_AS_OF_20250607,N_VOTERS_19_AS_OF_20250607,N_VOTERS_45_PLUS_AS_OF_20250607,EST_15_TO_17_YO,EST_17_OR_18_OR_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_RATE_18_YO_AS_OF_20250607,EST_REG_RATE_45_PLUS_YO_AS_OF_20250607
82,183,WAKE,4087,13339,431642,47558,15852.666667,499266,0.257812,0.864553
14,119,MECKLENBURG,3431,11411,391201,42911,14303.666667,465789,0.239869,0.839867
7,081,GUILFORD,1499,6330,210353,21093,7031.000000,253633,0.213199,0.82936
81,067,FORSYTH,1129,4319,149611,15559,5186.333333,186737,0.217688,0.801186
47,021,BUNCOMBE,730,2176,123456,8663,2887.666667,144295,0.252799,0.855581
...,...,...,...,...,...,...,...,...,...,...
15,103,JONES,17,79,4544,392,130.666667,5813,0.130102,0.781696
38,029,CAMDEN,48,122,4888,480,160.000000,5604,0.3,0.872234
71,075,GRAHAM,30,74,4031,297,99.000000,4616,0.30303,0.873267
65,095,HYDE,6,33,2123,121,40.333333,2830,0.14876,0.750177


#### Output
Write back to BQ

In [13]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_county_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


### Statewide Scorecard

In [14]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
acs_S0101_table = "S0101_us_states_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file for NY, counting the number of registrants at a given age
* Then, left joins the state estimates for the total number of 17, 18, and 19yos from the ACS
    * The estimates for the total number of 17-19yo yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

* Ages are calculated as of the date of the voter file and as of the next general election 

In [15]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH young_voters AS (
  SELECT  
    *,
    FLOOR(DATE_DIFF(
  DATE '2025-06-07', 
  DATE(CAST(YEAR_OF_BIRTH AS INT64), 1, 1), 
  DAY
  
) / 365.25) AS AGE_TODAY
  FROM `nc_production.20250607_scorecard_nc`
  WHERE VOTER_STATUS IN ('ACTIVE', 'INACTIVE')

), young_voter_sum AS(

SELECT
"37" AS STATE_FIPS ,
COUNTIF(AGE_TODAY = 18) AS N_VOTERS_18_AS_OF_""" + data_date_suffix + """,
COUNTIF(AGE_TODAY = 19) AS N_VOTERS_19_AS_OF_""" + data_date_suffix + """,
COUNTIF(AGE_TODAY >= 45) AS N_VOTERS_45_PLUS_AS_OF_""" + data_date_suffix + """,
FROM young_voters
GROUP BY STATE_FIPS

), acs_state AS(

SELECT 
STATE_FIPS,
EST_15_TO_17_YO,
EST_15_TO_17_YO / 3 AS  """ + EST_17_OR_18_OR_19_YO_THIS_YEAR + """,
EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
FROM `tcc-research.acs_sources.S0101_us_states_acs5y_2022` 
WHERE STATE_FIPS = "37"

)

SELECT
young_voter_sum.*,
acs_state.EST_15_TO_17_YO,
acs_state.""" + EST_17_OR_18_OR_19_YO_THIS_YEAR + """,
acs_state.""" + EST_45_PLUS_YO_THIS_YEAR + """,
FROM young_voter_sum LEFT JOIN acs_state ON young_voter_sum.STATE_FIPS = acs_state.STATE_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [16]:
# Preview
df.head()

,STATE_FIPS,N_VOTERS_18_AS_OF_20250607,N_VOTERS_19_AS_OF_20250607,N_VOTERS_45_PLUS_AS_OF_20250607,EST_15_TO_17_YO,EST_17_OR_18_OR_19_YO_2025,EST_45_PLUS_YO_2025
0,37,31790,110816,4308218,404849,134949.666667,5139416


In [17]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)


In [18]:
# Calculate estimated registration rate

N_VOTERS_18_AS_OF_DATA_DATE = 'N_VOTERS_18_AS_OF_'+ data_date_suffix # col name for estimated 18yo voters as of rolling date


EST_REG_RATE_18_YO_AS_OF_DATA_DATE = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

df_reg_est[EST_REG_RATE_18_YO_AS_OF_DATA_DATE] = df_reg_est[N_VOTERS_18_AS_OF_DATA_DATE] / df_reg_est[EST_17_OR_18_OR_19_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

df_reg_est

,STATE_FIPS,N_VOTERS_18_AS_OF_20250607,N_VOTERS_19_AS_OF_20250607,N_VOTERS_45_PLUS_AS_OF_20250607,EST_15_TO_17_YO,EST_17_OR_18_OR_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_RATE_18_YO_AS_OF_20250607
0,37,31790,110816,4308218,404849,134949.666667,5139416,0.235569


#### Metric 3: Estimated registration rate of 45+ year olds as of a rolling date (i.e. latest month)


In [19]:
# Calculate estimated registration rate

N_VOTERS_45_PLUS_AS_OF_DATA_DATE = 'N_VOTERS_45_PLUS_AS_OF_'+ data_date_suffix # col name for estimated 18yo voters as of rolling date

EST_REG_RATE_45_PLUS_YO_AS_OF_DATA_DATE = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_DATA_DATE] = df_reg_est[N_VOTERS_45_PLUS_AS_OF_DATA_DATE] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

df_reg_est.sort_values(EST_45_PLUS_YO_THIS_YEAR, ascending=False)

,STATE_FIPS,N_VOTERS_18_AS_OF_20250607,N_VOTERS_19_AS_OF_20250607,N_VOTERS_45_PLUS_AS_OF_20250607,EST_15_TO_17_YO,EST_17_OR_18_OR_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_RATE_18_YO_AS_OF_20250607,EST_REG_RATE_45_PLUS_YO_AS_OF_20250607
0,37,31790,110816,4308218,404849,134949.666667,5139416,0.235569,0.83827


#### Output
Write back to BQ

In [20]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_statewide_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
